In [16]:
# =============================================
# Laboratorio: Análisis Tabular con pandas y DuckDB
# =============================================

# 0) Preparación: instalar librerías
!pip install pandas duckdb pyarrow --quiet

import pandas as pd
import duckdb

# =============================================
# 1) Parte A — pandas
# =============================================

# A.1 Lectura y exploración
csv_path = "/content/20th century deaths in US - CDC.csv"

# Leer CSV
df = pd.read_csv(csv_path)

# Limpiar nombres de columnas (quitar espacios al inicio/final)
df.columns = df.columns.str.strip()

# Mostrar primeras filas
print("Primeras filas del dataset:")
display(df.head())

# Información general
print("\nInformación general:")
df.info()

# Conteo de nulos por columna
print("\nNulos por columna:")
print(df.isna().sum())

# Número de filas y columnas
print(f"\nDimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")

# =============================================
# A.2 Columnas derivadas y limpieza
# =============================================

# Crear columna derivada: DeathRatePer100k usando 'Total deaths  - Deaths' y 'Population' si existe
# Nota: tu CSV no tiene columna 'Population', por lo que este paso se omite si no existe

# Gestionar nulos: rellenar nulos de 'Total deaths  - Deaths' con 0
df['Total deaths  - Deaths'] = df['Total deaths  - Deaths'].fillna(0)

print("\nDataset tras limpieza de nulos:")
display(df.head())

# =============================================
# A.3 Agrupaciones y filtros
# =============================================

# Agrupar por 'Entity' y calcular métricas sobre 'Total deaths  - Deaths'
resumen = df.groupby('Entity').agg(
    TotalDeaths=('Total deaths  - Deaths', 'sum'),
    AvgDeaths=('Total deaths  - Deaths', 'mean')
).reset_index()

print("\nResumen agrupado por Entity:")
display(resumen.head())

# Exportar a CSV
resumen.to_csv("/content/resumen_pandas.csv", index=False)
print("Resumen pandas exportado a 'resumen_pandas.csv'")

# =============================================
# 2) Parte B — DuckDB
# =============================================

# Crear conexión DuckDB en memoria
con = duckdb.connect(database=':memory:')

# B.1 Primer query sobre CSV
query1 = f"""
SELECT Entity, COUNT(*) AS NumRows
FROM '{csv_path}'
GROUP BY Entity
ORDER BY NumRows DESC
LIMIT 5
"""
print("\nTop 5 entidades por número de registros (DuckDB):")
display(con.execute(query1).df())

# B.2 Repetir análisis en SQL
query2 = f"""
SELECT
    Entity,
    SUM("Total deaths  - Deaths") AS TotalDeaths,
    AVG("Total deaths  - Deaths") AS AvgDeaths
FROM '{csv_path}'
GROUP BY Entity
ORDER BY TotalDeaths DESC
"""
res_duckdb = con.execute(query2).df()
print("\nResumen DuckDB:")
display(res_duckdb.head())

# Exportar a CSV desde DuckDB
con.execute("COPY ( " + query2 + " ) TO '/content/resumen_duckdb.csv' WITH (HEADER TRUE)")
print("Resumen DuckDB exportado a 'resumen_duckdb.csv'")

# =============================================
# 3) Comparación breve
# =============================================
print("""
Comparación pandas vs DuckDB:
- pandas: más flexible, directo en Python, fácil manipulación y gráficos.
- DuckDB: SQL embebido, rápido en datasets grandes, directo desde archivos CSV sin cargar todo en memoria.
""")


Primeras filas del dataset:


,Entity,Year,Accidents (excl. road) - Deaths,Accidents (total) - Deaths,Arteriosclerosis - Deaths,Bronchitis - Deaths,Cancers - Deaths,COPD - Deaths,Dementia - Deaths,Diabetes - Deaths,...,Respiratory disease - % of Deaths,Road accidents - % of Deaths,Septicemia - % of Deaths,Stroke - % of Deaths,Suicide - % of Deaths,Syphilis - % of Deaths,Total deaths - % of Deaths,Tuberculosis - % of Deaths,Non-communicable diseases (NCDs) - % of Deaths,"Communicable, infectious, neonatal and other deaths - % of Deaths"
0,United States,1900,14429.0,14429.0,NaN,NaN,12769,NaN,10015.0,NaN,...,NaN,NaN,NaN,6.22,NaN,NaN,100.0,11.31,17.93,77.86
1,United States,1901,16958.0,NaN,NaN,8171.0,13438,NaN,9771.0,NaN,...,NaN,NaN,NaN,6.51,NaN,NaN,100.0,11.57,19.09,75.81
2,United States,1902,14915.0,NaN,NaN,8112.0,13653,NaN,9309.0,NaN,...,NaN,NaN,NaN,6.71,NaN,NaN,100.0,11.25,20.39,74.93
3,United States,1903,17050.0,NaN,NaN,7508.0,14650,NaN,8604.0,NaN,...,NaN,NaN,NaN,6.73,NaN,NaN,100.0,11.34,20.92,73.87
4,United States,1904,18228.0,NaN,NaN,7571.0,15247,NaN,8702.0,NaN,...,NaN,NaN,NaN,6.62,NaN,NaN,100.0,11.47,20.96,73.83



Información general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 84 columns):
 #   Column                                                             Non-Null Count  Dtype  
---  ------                                                             --------------  -----  
 0   Entity                                                             94 non-null     object 
 1   Year                                                               94 non-null     int64  
 2   Accidents (excl. road) - Deaths                                    92 non-null     float64
 3   Accidents (total)  - Deaths                                        72 non-null     float64
 4   Arteriosclerosis  - Deaths                                         40 non-null     float64
 5   Bronchitis  - Deaths                                               11 non-null     float64
 6   Cancers  - Deaths                                                  94 non-null     int64  
 7   COPD  

,Entity,Year,Accidents (excl. road) - Deaths,Accidents (total) - Deaths,Arteriosclerosis - Deaths,Bronchitis - Deaths,Cancers - Deaths,COPD - Deaths,Dementia - Deaths,Diabetes - Deaths,...,Respiratory disease - % of Deaths,Road accidents - % of Deaths,Septicemia - % of Deaths,Stroke - % of Deaths,Suicide - % of Deaths,Syphilis - % of Deaths,Total deaths - % of Deaths,Tuberculosis - % of Deaths,Non-communicable diseases (NCDs) - % of Deaths,"Communicable, infectious, neonatal and other deaths - % of Deaths"
0,United States,1900,14429.0,14429.0,NaN,NaN,12769,NaN,10015.0,NaN,...,NaN,NaN,NaN,6.22,NaN,NaN,100.0,11.31,17.93,77.86
1,United States,1901,16958.0,NaN,NaN,8171.0,13438,NaN,9771.0,NaN,...,NaN,NaN,NaN,6.51,NaN,NaN,100.0,11.57,19.09,75.81
2,United States,1902,14915.0,NaN,NaN,8112.0,13653,NaN,9309.0,NaN,...,NaN,NaN,NaN,6.71,NaN,NaN,100.0,11.25,20.39,74.93
3,United States,1903,17050.0,NaN,NaN,7508.0,14650,NaN,8604.0,NaN,...,NaN,NaN,NaN,6.73,NaN,NaN,100.0,11.34,20.92,73.87
4,United States,1904,18228.0,NaN,NaN,7571.0,15247,NaN,8702.0,NaN,...,NaN,NaN,NaN,6.62,NaN,NaN,100.0,11.47,20.96,73.83



Resumen agrupado por Entity:


...

Resumen pandas exportado a 'resumen_pandas.csv'

Top 5 entidades por número de registros (DuckDB):


,Entity,NumRows
0,United States,94



Resumen DuckDB:


,Entity,TotalDeaths,AvgDeaths
0,United States,140507665.0,1.494762e+06


Resumen DuckDB exportado a 'resumen_duckdb.csv'

Comparación pandas vs DuckDB:
- pandas: más flexible, directo en Python, fácil manipulación y gráficos.
- DuckDB: SQL embebido, rápido en datasets grandes, directo desde archivos CSV sin cargar todo en memoria.

